In [8]:
# average number of OSM mappers per months since 2022
start_year = 2023
data_path = '/data/processing/partitioned-ohsome-parquet-100997/*/*/*.parquet'

sql = f"""
COPY
(
    SELECT
        country_iso_a3,
        year,
        count(distinct user_id) as n_users
        count()
    FROM (
    	SELECT user_id, unnest(country_iso_a3) as country_iso_a3, year
    	FROM read_parquet("{data_path}", hive_partitioning=1)
    	WHERE year = {start_year}
    ) foo
    GROUP BY year, country_iso_a3
    ORDER BY country_iso_a3, year
    )
TO 'mappers_per_country_2023.csv' (HEADER, DELIMITER ',');
"""
print(sql)


COPY
(
    SELECT
        country_iso_a3,
        year,
        count(distinct user_id) as n_users
        count()
    FROM (
    	SELECT user_id, unnest(country_iso_a3) as country_iso_a3, year
    	FROM read_parquet("/data/processing/partitioned-ohsome-parquet-100997/*/*/*.parquet", hive_partitioning=1)
    	WHERE year = 2023
    ) foo
    GROUP BY year, country_iso_a3
    ORDER BY country_iso_a3, year
    )
TO 'mappers_per_country_2023.csv' (HEADER, DELIMITER ',');



In [22]:
import requests

def get_team_members(team_id):
    page_number = 1
    max_pages = 0
    user_ids = []
    user_names = []
    
    while ((page_number == 1) or (page_number <= max_pages)):
        url = f"https://mapping.team/api/teams/{team_id}/members"
        data = requests.get(url, params={"page": page_number}).json()

        team_members = data["members"]["data"]
        max_pages = data["members"]["pagination"]["lastPage"]

        user_ids += [item["id"] for item in team_members]
        user_names += [item["name"] for item in team_members]
        page_number += 1
        
    return user_ids, user_names

# team id 1484 --> youthmappers summit participants
user_ids, user_names = get_team_members(1484)
print(user_ids)
print(user_names)

[20115162, 13426455, 15839807, 17603092, 10858729, 16266664, 16060450, 18410224, 17191461, 18357124, 1770239, 14480656, 17582418, 14854690, 17014167, 20252879, 14334590, 16477254, 15741221, 14645740, 13203366, 2913188, 16119062, 16845930, 16815949]
['adamutafidah', 'Alex Buenaventura', 'Alvin1', 'Anne Marie Vic Lobo', 'ArtzaiPY', 'Brazil Singh', 'Dibikshya Shrestha', 'Hyacinth_Jane', 'Ivy Perez', 'JaneAmerica12', 'Jennings Anderson', 'joyceeemaeee', 'Julia Teixeira', 'Lochan Pant', 'Maluleke Karabo vinolia', 'MariaTimba', 'Mary Muthee', 'Marzia_Hossain', 'Sajeevini sivajothy', 'Samuel Ackon', 'Sangeeta BC', 'Sawan Shariar', 'seejahin', 'SilvEsth', 'Zuliatu']


In [3]:
user_ids = [20115162, 13426455, 15839807, 17603092, 10858729, 16266664, 16060450, 18410224, 17191461, 18357124, 1770239, 14480656, 17582418, 14854690, 17014167, 20252879, 14334590, 16477254, 15741221, 14645740, 13203366, 2913188, 16119062, 16845930, 16815949]
print(user_ids)

[20115162, 13426455, 15839807, 17603092, 10858729, 16266664, 16060450, 18410224, 17191461, 18357124, 1770239, 14480656, 17582418, 14854690, 17014167, 20252879, 14334590, 16477254, 15741221, 14645740, 13203366, 2913188, 16119062, 16845930, 16815949]


In [24]:
sql = f"""
COPY (
    SELECT
        user_id,
        year,
        count(*) as n_edits,
        Sum(CASE
            WHEN building_edit = 1 THEN 1
            ELSE 0
        END) as n_buildings_created,
        Sum(CASE
            WHEN building_edit = 0 THEN 1
            ELSE 0
        END) as n_buildings_modified,
        Sum(CASE
            WHEN building_edit = -1 THEN 1
            ELSE 0
        END) as n_buildings_deleted,
        sum(building_edit) as delta_buildings
    FROM read_parquet("{data_path}", hive_partitioning=1)
    WHERE 1=1
        AND year = {start_year}
        AND user_id IN (20115162, 13426455, 15839807, 17603092, 10858729, 16266664, 16060450, 18410224, 17191461, 18357124, 1770239, 14480656, 17582418, 14854690, 17014167, 20252879, 14334590, 16477254, 15741221, 14645740, 13203366, 2913188, 16119062, 16845930, 16815949)
    GROUP BY user_id, year
    ORDER BY user_id
)
TO 'user_stats_2023.csv' (HEADER, DELIMITER ',');
"""
print(sql)


COPY (
    SELECT
        user_id,
        year,
        count(*) as n_edits,
        Sum(CASE
            WHEN building_edit = 1 THEN 1
            ELSE 0
        END) as n_buildings_created,
        Sum(CASE
            WHEN building_edit = 0 THEN 1
            ELSE 0
        END) as n_buildings_modified,
        Sum(CASE
            WHEN building_edit = -1 THEN 1
            ELSE 0
        END) as n_buildings_deleted,
        sum(building_edit) as delta_buildings
    FROM read_parquet("/data/processing/partitioned-ohsome-parquet-100997/*/*/*.parquet", hive_partitioning=1)
    WHERE 1=1
        AND year = 2023
        AND user_id IN (20115162, 13426455, 15839807, 17603092, 10858729, 16266664, 16060450, 18410224, 17191461, 18357124, 1770239, 14480656, 17582418, 14854690, 17014167, 20252879, 14334590, 16477254, 15741221, 14645740, 13203366, 2913188, 16119062, 16845930, 16815949)
    GROUP BY user_id, year
    ORDER BY user_id
)
TO 'user_stats_2023.csv' (HEADER, DELIMITER ',');



In [46]:
# retention rate

sql = f"""
CREATE TABLE user_mapping_stats AS
SELECT
    user_id,
    count(*) as n_edits,
    min(date_trunc('day', epoch_ms(changeset_timestamp))) as min_timestamp,
    max(date_trunc('day', epoch_ms(changeset_timestamp))) as max_timestamp,
    count(distinct date_trunc('day', epoch_ms(changeset_timestamp))) as n_days,
    min(changeset_id) as first_changeset
FROM read_parquet("{data_path}", hive_partitioning=1)
GROUP BY user_id
HAVING min_timestamp >= '2023-01-01';

CREATE TABLE changesets_2023 AS
SELECT
    changeset_id,
    hashtags
FROM read_parquet("{data_path}", hive_partitioning=1)
WHERE epoch_ms(changeset_timestamp) >= '2023-01-01'
GROUP BY changeset_id, hashtags;
    
COPY (
    SELECT 
        a.*,
        b.hashtags,
        list_has_any(b.hashtags, ['missingmaps', 'MissingMaps', 'missing_maps']) as missing_maps,
        list_contains(list_transform(b.hashtags, x -> x like 'hotosm-project-%'), true) as hot_tasking_manager,
        CASE
            WHEN user_id IN (27099, 54231, 55024, 82774, 128569, 171231, 178521, 191308, 315995, 366834, 386103, 393098, 393359, 408769, 507262, 561750, 563706, 563947, 593765, 604227, 604538, 643900, 673875, 676762, 918955, 1050248, 1210319, 1217047, 1521733, 1535264, 1639773, 1725054, 1727004, 1769935, 1770239, 1778694, 1779108, 1793105, 1803680, 1803854, 1902578, 1915192, 1935178, 1957873, 1964962, 1998982, 2022154, 2052228, 2111087, 2130214, 2132030, 2150621, 2187425, 2204318, 2245758, 2248202, 2280594, 2327399, 2327410, 2327498, 2327521, 2329822, 2330248, 2331874, 2385561, 2405982, 2589926, 2593307, 2605569, 2623654, 2624064, 2732604, 2757453, 2783807, 2789434, 2789436, 2803704, 2815311, 2815313, 2816546, 2819243, 2824952, 2828866, 2862016, 2869420, 2874300, 2913188, 2913194, 2913504, 2914467, 2947555, 2975705, 3042083, 3044490, 3046993, 3050032, 3112087, 3113029, 3115457, 3116033, 3116122, 3117261, 3117282, 3117626, 3125906, 3127011, 3180585, 3184504, 3236991, 3291762, 3303302, 3338106, 3345135, 3345775, 3349119, 3363190, 3365317, 3367229, 3367241, 3367243, 3367245, 3367248, 3367251, 3367252, 3367313, 3369319, 3376856, 3376862, 3381287, 3381374, 3391305, 3396014, 3397290, 3400042, 3401338, 3407771, 3455921, 3471975, 3481032, 3481073, 3485020, 3502753, 3502761, 3502769, 3502771, 3502824, 3502845, 3504693, 3505733, 3509533, 3512474, 3512495, 3512550, 3512887, 3520331, 3520401, 3520403, 3522458, 3524470, 3536908, 3548727, 3548733, 3551459, 3553162, 3553170, 3553481, 3565118, 3573216, 3575045, 3575055, 3580954, 3582506, 3588829, 3592102, 3592184, 3593219, 3593426, 3596562, 3601934, 3608418, 3609913, 3609914, 3625162, 3627450, 3627707, 3627995, 3628443, 3628506, 3628546, 3628921, 3629560, 3630974, 3632982, 3633498, 3634256, 3636462, 3636468, 3636884, 3639250, 3642288, 3645531, 3648428, 3649667, 3650980, 3660849, 3660854, 3668314, 3671339, 3682432, 3686622, 3686666, 3693601, 3695164, 3734102, 3737095, 3748269, 3767710, 3769220, 3769454, 3769470, 3794884, 3794900, 3796596, 3830109, 3833584, 3851725, 3851730, 3851737, 3851749, 3854561, 3856913, 3859901, 3878185, 3886087, 3888722, 3888755, 3888812, 3888908, 3889552, 3930537, 3934069, 3951928, 3975528, 3996686, 4010972, 4011403, 4015712, 4023437, 4039113, 4063492, 4110434, 4149748, 4202290, 4223917, 4316098, 4319101, 4352853, 4366372, 4366421, 4366422, 4366434, 4366436, 4379874, 4419442, 4434231, 4434270, 4456266, 4456639, 4456643, 4456655, 4456657, 4456665, 4456732, 4458831, 4459284, 4459720, 4459793, 4466190, 4466195, 4466209, 4467310, 4470074, 4470083, 4471622, 4471623, 4471624, 4471625, 4471628, 4471629, 4471643, 4471655, 4471851, 4476246, 4479975, 4516636, 4516652, 4516666, 4516681, 4516684, 4516743, 4516804, 4516830, 4517943, 4518610, 4518760, 4521382, 4530401, 4530956, 4530972, 4530980, 4531021, 4531045, 4531059, 4531060, 4531377, 4533410, 4541953, 4542220, 4543032, 4543231, 4543255, 4543524, 4543600, 4543850, 4543864, 4543868, 4545114, 4545134, 4545135, 4545193, 4545213, 4545231, 4545678, 4545852, 4550348, 4550903, 4550905, 4551656, 4551660, 4551663, 4551664, 4551682, 4551705, 4551931, 4556176, 4563312, 4563883, 4564910, 4564929, 4569905, 4576568, 4577601, 4582659, 4585394, 4587793, 4587800, 4589870, 4610319, 4610340, 4610363, 4610468, 4610568, 4610711, 4610725, 4621166, 4621346, 4621352, 4630994, 4631420, 4645012, 4645947, 4653136, 4653441, 4653773, 4655090, 4655091, 4655093, 4655096, 4655115, 4655120, 4672373, 4672528, 4681094, 4685020, 4690780, 4694285, 4714195, 4719316, 4723292, 4723377, 4737801, 4742359, 4750403, 4750411, 4750414, 4750466, 4750581, 4755369, 4755418, 4755435, 4755754, 4755768, 4755790, 4780240, 4795315, 4795320, 4795323, 4795332, 4795340, 4795347, 4795375, 4795381, 4795397, 4795416, 4795420, 4795500, 4795506, 4797131, 4797156, 4797177, 4797229, 4804704, 4810380, 4810482, 4810484, 4810485, 4814747, 4814822, 4814987, 4816007, 4816114, 4825148, 4825748, 4826501, 4828025, 4829257, 4831172, 4831302, 4834302, 4835663, 4836584, 4837357, 4837361, 4837426, 4838813, 4838903, 4846853, 4849501, 4853368, 4853369, 4853701, 4853783, 4853807, 4853808, 4853836, 4853839, 4853859, 4853867, 4854138, 4854623, 4854783, 4854861, 4854939, 4857822, 4858154, 4858652, 4860449, 4861141, 4861563, 4861595, 4861681, 4861695, 4861938, 4862041, 4862042, 4862063, 4862072, 4862228, 4863679, 4863686, 4863699, 4863848, 4873392, 4873427, 4873431, 4873450, 4873470, 4874033, 4874102, 4887075, 4899670, 4908508, 4909982, 4928257, 4950300, 4971333, 4977551, 4978827, 5010433, 5022087, 5037631, 5039176, 5093311, 5112404, 5117109, 5129203, 5136132, 5136182, 5141416, 5147024, 5149809, 5153612, 5154063, 5179309, 5188606, 5191095, 5192895, 5193350, 5195345, 5195536, 5198564, 5209776, 5209801, 5209819, 5209824, 5212877, 5227090, 5236091, 5236947, 5239509, 5240907, 5254921, 5257863, 5266174, 5267631, 5270698, 5275155, 5275165, 5275678, 5278057, 5281513, 5290000, 5291232, 5294434, 5294450, 5295021, 5298447, 5298569, 5299988, 5307929, 5310527, 5310986, 5316526, 5319796, 5319799, 5319805, 5319814, 5319817, 5320010, 5320013, 5320021, 5320024, 5320249, 5320264, 5320413, 5323277, 5323284, 5323285, 5323296, 5323301, 5323310, 5323336, 5326054, 5326154, 5328123, 5329058, 5330232, 5330990, 5331092, 5331204, 5331225, 5331297, 5331690, 5332045, 5337025, 5337097, 5337261, 5340340, 5340528, 5365592, 5374523, 5377023, 5379313, 5379337, 5392008, 5421563, 5422174, 5444270, 5448960, 5449028, 5455971, 5466476, 5466597, 5467159, 5484092, 5484094, 5484300, 5484336, 5484541, 5489383, 5545936, 5562404, 5578399, 5585721, 5596182, 5615669, 5615898, 5624644, 5624958, 5625355, 5627718, 5629648, 5646866, 5655451, 5658513, 5659317, 5659378, 5659715, 5689003, 5696137, 5696198, 5702683, 5707143, 5708406, 5718341, 5726905, 5726933, 5748265, 5748734, 5752250, 5752629, 5854692, 5863675, 5867004, 5867434, 5867704, 5881052, 5891083, 5897965, 5921856, 5942196, 5942517, 5942719, 5942862, 5943155, 5943289, 5943826, 6000145, 6000163, 6000171, 6000181, 6000183, 6000211, 6000380, 6001174, 6001440, 6017386, 6058846, 6067659, 6067668, 6074987, 6075008, 6075030, 6075038, 6075041, 6075053, 6075257, 6106947, 6111679, 6125882, 6126808, 6136314, 6150323, 6163141, 6163623, 6173770, 6175560, 6175966, 6176290, 6177995, 6179768, 6192228, 6192530, 6192851, 6195345, 6200023, 6201821, 6201856, 6201866, 6202367, 6202387, 6204667, 6206544, 6207019, 6211207, 6212630, 6212762, 6212799, 6213019, 6213371, 6233686, 6234449, 6237421, 6239474, 6239511, 6239541, 6239577, 6239855, 6241856, 6242902, 6243439, 6246337, 6249131, 6249877, 6249886, 6249908, 6249923, 6306737, 6307214, 6309404, 6312144, 6312212, 6313846, 6317343, 6322650, 6323155, 6323172, 6323223, 6323411, 6329771, 6334327, 6337421, 6342184, 6342189, 6342214, 6342278, 6352785, 6359620, 6362084, 6405430, 6406545, 6411973, 6415223, 6465366, 6465381, 6465429, 6465623, 6465640, 6465660, 6465671, 6465688, 6465700, 6465713, 6465978, 6466697, 6474298, 6475519, 6481304, 6503849, 6503859, 6503872, 6507231, 6509614, 6509998, 6520163, 6520256, 6520301, 6528197, 6528213, 6528235, 6529513, 6529630, 6529746, 6529792, 6530414, 6533575, 6534235, 6535064, 6535648, 6535722, 6535879, 6539988, 6540873, 6545197, 6546372, 6546934, 6547685, 6549191, 6549194, 6552110, 6555839, 6556202, 6557104, 6557317, 6557448, 6557742, 6559755, 6599895, 6606782, 6636249, 6636922, 6638443, 6643932, 6648710, 6652088, 6652179, 6653283, 6660054, 6660094, 6660119, 6660144, 6660182, 6660191, 6661941, 6665480, 6674701, 6675564, 6677927, 6680704, 6680715, 6681057, 6688997, 6689154, 6689453, 6689456, 6689921, 6689931, 6696681, 6696709, 6711663, 6713759, 6715932, 6720602, 6723048, 6726590, 6726596, 6726600, 6726607, 6726616, 6726619, 6726623, 6726647, 6726661, 6726672, 6726677, 6726686, 6726693, 6726702, 6726711, 6726737, 6726738, 6733614, 6734546, 6746561, 6753040, 6755994, 6791097, 6791625, 6793349, 6793356, 6795786, 6795789, 6797474, 6797922, 6802216, 6806919, 6812135, 6812140, 6812144, 6812353, 6819172, 6829138, 6831676, 6831973, 6841161, 6841213, 6842923, 6866977, 6873995, 6882856, 6891510, 6893434, 6893771, 6893935, 6895044, 6895483, 6903002, 6903073, 6903135, 6935059, 6935263, 6950137, 6962560, 6962787, 6962791, 6962796, 6962813, 6962827, 6962837, 6967037, 6968212, 6968661, 6980380, 6980804, 6980806, 6980809, 6980813, 6980816, 6980818, 6980820, 6980821, 6980824, 6981028, 6981031, 6981033, 6981036, 6981041, 6981046, 6981047, 6981048, 6981052, 6981053, 6981056, 6982496, 6984710, 6987266, 6989772, 6989797, 6989799, 6989803, 6989928, 6989983, 6990048, 6990114, 6990860, 6990887, 6991194, 6991874, 6991886, 6991897, 6992012, 6992840, 6994816, 6997855, 6997856, 6997863, 6997866, 6997867, 6997950, 6997999, 6998065, 6998399, 6998858, 7001461, 7001522, 7001528, 7001606, 7002112, 7009028, 7009378, 7009393, 7009482, 7009574, 7009576, 7009578, 7009582, 7009584, 7009585, 7009586, 7009587, 7009588, 7009590, 7009592, 7009594, 7009595, 7009597, 7009603, 7009657, 7009794, 7010032, 7034610, 7053510, 7063886, 7115639, 7115972, 7124841, 7131883, 7154329, 7159298, 7162262, 7164287, 7169521, 7179226, 7179671, 7214903, 7217628, 7233400, 7242430, 7242581, 7243359, 7243426, 7243605, 7254945, 7255043, 7255659, 7354300, 7410158, 7447450, 7516714, 7529952, 7533975, 7567509, 7599517, 7600327, 7600329, 7606880, 7622589, 7624135, 7631742, 7633451, 7657619, 7658859, 7666560, 7666645, 7666726, 7681679, 7689102, 7689176, 7692594, 7692596, 7692631, 7692654, 7692656, 7692663, 7692664, 7692666, 7692679, 7692681, 7692684, 7692688, 7692693, 7692695, 7692969, 7703465, 7708747, 7709395, 7710394, 7713992, 7714106, 7734979, 7777526, 7795270, 7823217, 7834299, 7858115, 7858616, 7911952, 7911964, 7912018, 7912243, 7912251, 7912256, 7912257, 7912263, 7912298, 7912312, 7912315, 7949960, 7950417, 7964630, 7964812, 7966705, 7977256, 8102301, 8102312, 8110709, 8138348, 8155627, 8220038, 8233250, 8253524, 8277193, 8288535, 8288596, 8292606, 8292630, 8292649, 8292657, 8292727, 8293676, 8297568, 8309360, 8310862, 8311514, 8315456, 8316170, 8317296, 8317902, 8330489, 8333271, 8353419, 8377453, 8380708, 8386618, 8389251, 8392175, 8413832, 8414406, 8425742, 8431311, 8472066, 8474278, 8474299, 8474365, 8474575, 8478755, 8484471, 8502322, 8514452, 8521341, 8526180, 8535118, 8535574, 8543379, 8553006, 8555267, 8557611, 8557815, 8564211, 8564467, 8576693, 8593936, 8609412, 8626264, 8640661, 8640714, 8640750, 8668411, 8683526, 8684339, 8693193, 8698377, 8733281, 8733294, 8734324, 8734332, 8738512, 8747689, 8749272, 8772532, 8773807, 8773991, 8774532, 8801128, 8809024, 8822271, 8825311, 8825695, 8825750, 8827882, 8833032, 8836427, 8852922, 8852953, 8853135, 8857309, 8857496, 8871263, 8871486, 8871772, 8874443, 8879119, 8880467, 8881365, 8882421, 8883831, 8884106, 8884110, 8884245, 8885047, 8885056, 8885127, 8885135, 8885248, 8886943, 8887664, 8889983, 8890000, 8899303, 8930627, 8934031, 8971246, 8991600, 8992397, 9016554, 9019904, 9021289, 9028962, 9028970, 9039501, 9054148, 9059201, 9059245, 9060686, 9060689, 9060690, 9060692, 9063710, 9063754, 9064751, 9067142, 9067419, 9090183, 9093600, 9094296, 9094298, 9094303, 9094312, 9094337, 9097672, 9097678, 9097681, 9097700, 9097882, 9098075, 9125001, 9155658, 9160253, 9163481, 9167598, 9168467, 9202190, 9215960, 9223721, 9232250, 9234461, 9264274, 9264317, 9272311, 9342997, 9351084, 9356680, 9361895, 9362957, 9374154, 9383528, 9389700, 9394713, 9398759, 9398793, 9398865, 9402325, 9426620, 9426669, 9436646, 9446263, 9446269, 9446286, 9449447, 9449463, 9459426, 9462170, 9463670, 9467533, 9476543, 9476545, 9476547, 9486708, 9489244, 9489250, 9489296, 9489407, 9491276, 9492821, 9513771, 9513779, 9513805, 9522913, 9525499, 9530563, 9531596, 9533675, 9534424, 9544112, 9553534, 9553539, 9554224, 9555656, 9557083, 9570396, 9571250, 9572693, 9574037, 9574249, 9574432, 9574619, 9574624, 9574627, 9574629, 9574638, 9580890, 9580891, 9580898, 9580960, 9580964, 9602004, 9602857, 9619006, 9620670, 9622928, 9624650, 9624795, 9625064, 9636868, 9636881, 9640365, 9648272, 9926812, 9946255, 9946686, 9946801, 9964969, 9966136, 9966880, 9969873, 10000054, 10002113, 10019356, 10019754, 10019802, 10020481, 10020547, 10020576, 10021541, 10021808, 10030205, 10033761, 10044753, 10045809, 10047805, 10047837, 10062444, 10068373, 10068412, 10069125, 10071053, 10072236, 10073390, 10083341, 10110280, 10111729, 10118836, 10125196, 10126896, 10128086, 10137724, 10137775, 10138066, 10138238, 10138560, 10138757, 10138825, 10139406, 10139430, 10139623, 10140159, 10140184, 10140186, 10140227, 10140358, 10140402, 10140572, 10141068, 10141120, 10142645, 10150885, 10162073, 10162076, 10162079, 10162080, 10162083, 10162087, 10162089, 10162096, 10162097, 10162099, 10162100, 10162105, 10162109, 10162110, 10162111, 10162113, 10162120, 10162122, 10162536, 10163273, 10170056, 10174407, 10184312, 10186120, 10194382, 10194383, 10194384, 10194386, 10194387, 10194388, 10194389, 10194390, 10194391, 10194392, 10194393, 10194394, 10194398, 10194415, 10194430, 10196983, 10197080, 10205518, 10211495, 10218760, 10227210, 10228236, 10242484, 10245957, 10246198, 10249523, 10252701, 10252703, 10252708, 10252710, 10252714, 10252716, 10252720, 10252728, 10252731, 10252732, 10252737, 10252739, 10252752, 10252757, 10252761, 10252762, 10252792, 10253135, 10253243, 10254841, 10257494, 10257798, 10257929, 10264522, 10271734, 10279009, 10285872, 10294659, 10295058, 10296067, 10296332, 10297931, 10298376, 10298452, 10298667, 10299297, 10300870, 10300871, 10300875, 10300876, 10300877, 10300880, 10300881, 10300882, 10300888, 10300893, 10300895, 10300899, 10300907, 10300919, 10300995, 10301056, 10303316, 10303893, 10303940, 10305086, 10305121, 10306905, 10307099, 10307182, 10307196, 10307406, 10307836, 10307856, 10307860, 10307869, 10309225, 10309398, 10309693, 10310242, 10310563, 10310602, 10310605, 10310739, 10310781, 10312497, 10313455, 10314035, 10314775, 10314916, 10315213, 10315233, 10315245, 10315246, 10315286, 10315810, 10315817, 10315823, 10315824, 10315827, 10315844, 10315855, 10315899, 10315904, 10315915, 10316037, 10316295, 10317067, 10324752, 10325675, 10328649, 10332104, 10333952, 10334118, 10335385, 10335471, 10336280, 10344444, 10344453, 10348336, 10350923, 10350932, 10352738, 10352796, 10352870, 10353849, 10353875, 10353905, 10353909, 10353911, 10365273, 10365640, 10365739, 10366307, 10369293, 10371572, 10371825, 10375489, 10376870, 10377118, 10377934, 10380110, 10381183, 10381887, 10382376, 10382752, 10382754, 10389211, 10392282, 10392792, 10393901, 10393966, 10394070, 10394108, 10394138, 10394161, 10394186, 10394201, 10394700, 10394713, 10394714, 10394721, 10394726, 10394728, 10394732, 10394737, 10394772, 10394778, 10397517, 10397538, 10397553, 10401038, 10403792, 10408926, 10412037, 10412054, 10413303, 10413514, 10414983, 10414984, 10415004, 10415005, 10415006, 10415007, 10415008, 10415028, 10419753, 10422352, 10422389, 10422391, 10428097, 10428171, 10428195, 10428548, 10428870, 10429396, 10429461, 10431373, 10434543, 10444596, 10447784, 10447791, 10452562, 10453340, 10454980, 10459757, 10461053, 10465311, 10466134, 10466608, 10466749, 10466757, 10466768, 10469823, 10471556, 10472439, 10472441, 10472447, 10472550, 10473429, 10473724, 10476531, 10482675, 10483389, 10483390, 10483398, 10483411, 10483647, 10488582, 10491954, 10517772, 10553167, 10553190, 10569841, 10570996, 10572167, 10572533, 10575987, 10602213, 10602238, 10613104, 10613118, 10613138, 10618943, 10618978, 10618979, 10619016, 10641000, 10660787, 10665962, 10674298, 10683886, 10688519, 10697587, 10701038, 10723548, 10723557, 10723561, 10723563, 10723574, 10723577, 10723578, 10723697, 10734833, 10755763, 10766953, 10766954, 10766956, 10766962, 10766964, 10766969, 10766972, 10766973, 10766983, 10766991, 10766996, 10767006, 10767041, 10767042, 10767048, 10767067, 10768004, 10771699, 10771854, 10773950, 10774428, 10779819, 10784634, 10798401, 10805123, 10812819, 10812904, 10836586, 10837586, 10837709, 10845620, 10847467, 10855988, 10856000, 10856005, 10856028, 10856839, 10858729, 10864467, 10866497, 10866499, 10866500, 10866506, 10866571, 10866643, 10868392, 10868456, 10868648, 10876087, 10876133, 10882647, 10885730, 10885796, 10887171, 10894654, 10898257, 10921442, 10921645, 10934301, 10939841, 10963925, 10978697, 10979233, 10982439, 10984892, 10991352, 10997928, 11010087, 11010237, 11058007, 11084219, 11086590, 11137096, 11141773, 11158572, 11171572, 11171574, 11171592, 11171620, 11177499, 11178862, 11180925, 11295328, 11295416, 11297323, 11385528, 11395973, 11396063, 11396103, 11396143, 11432684, 11446724, 11491044, 11491048, 11491063, 11491064, 11491069, 11491073, 11491077, 11491080, 11491829, 11494405, 11512481, 11536988, 11539361, 11539365, 11539367, 11539368, 11539375, 11550159, 11566011, 11570880, 11572008, 11575121, 11585555, 11589556, 11589559, 11589561, 11589667, 11589867, 11600833, 11603914, 11632336, 11640369, 11640807, 11656047, 11656135, 11674896, 11730586, 11733358, 11774025, 11774060, 11779623, 11793207, 11807790, 11830680, 11835896, 11867310, 11868369, 11900841, 11908540, 11919491, 11943440, 11953893, 11976394, 11999420, 12015769, 12031941, 12067506, 12067647, 12072417, 12078651, 12088780, 12106999, 12109557, 12123439, 12129925, 12144247, 12151924, 12157812, 12176702, 12181999, 12184321, 12191928, 12193439, 12208589, 12208592, 12208593, 12208597, 12208603, 12208608, 12208610, 12208627, 12208662, 12209174, 12209177, 12209189, 12209195, 12209201, 12209216, 12209227, 12209478, 12210147, 12210356, 12210359, 12210417, 12210421, 12210501, 12210569, 12212203, 12212329, 12213080, 12213821, 12214057, 12214092, 12214577, 12214698, 12214787, 12215080, 12216668, 12231845, 12248057, 12248058, 12258107, 12263608, 12277006, 12280809, 12291129, 12311762, 12324205, 12339121, 12342156, 12357640, 12368208, 12378137, 12384423, 12384443, 12384464, 12384592, 12386161, 12387301, 12397323, 12400048, 12406019, 12406042, 12406070, 12406080, 12406088, 12407843, 12408162, 12424765, 12439450, 12441709, 12454484, 12480255, 12484304, 12520176, 12529516, 12531530, 12533960, 12534004, 12545625, 12547895, 12567517, 12572606, 12572714, 12572819, 12575202, 12595407, 12599925, 12615464, 12615466, 12615482, 12620938, 12629068, 12632505, 12640881, 12650130, 12651631, 12666546, 12691149, 12694216, 12696088, 12696819, 12698223, 12748853, 12774177, 12791691, 12796222, 12797808, 12798188, 12800129, 12810081, 12810545, 12810570, 12810572, 12822692, 12827111, 12829874, 12831597, 12847852, 12853146, 12853242, 12853574, 12860486, 12865158, 12865348, 12867442, 12867476, 12867509, 12879973, 12880236, 12880237, 12883224, 12883277, 12883311, 12908085, 12911941, 12918215, 12970584, 12993219, 13000502, 13005097, 13005199, 13005253, 13007399, 13013984, 13014934, 13015130, 13022853, 13023273, 13023274, 13023382, 13031108, 13031156, 13031777, 13039530, 13063544, 13063701, 13067323, 13076013, 13083632, 13086886, 13095983, 13095994, 13110187, 13114924, 13115486, 13117996, 13119139, 13119640, 13119957, 13120060, 13120394, 13123824, 13125074, 13125190, 13125269, 13125293, 13125421, 13140436, 13149325, 13158337, 13158452, 13158530, 13158654, 13158927, 13158938, 13159018, 13159052, 13159097, 13159413, 13162213, 13162222, 13163024, 13163038, 13163041, 13163043, 13163061, 13163078, 13163089, 13163096, 13163111, 13163116, 13163249, 13163363, 13164975, 13164985, 13165036, 13165051, 13165456, 13165468, 13165920, 13166458, 13170728, 13172373, 13180237, 13184051, 13203366, 13205258, 13205671, 13206115, 13207766, 13213172, 13214115, 13246372, 13248604, 13252977, 13253207, 13253583, 13255729, 13255920, 13261176, 13301985, 13322443, 13330486, 13352087, 13369867, 13377221, 13395522, 13401384, 13426455, 13426465, 13426470, 13426490, 13426512, 13442069, 13442272, 13442281, 13442487, 13443059, 13481812, 13529725, 13594329, 13594657, 13600653, 13639411, 13645764, 13685990, 13703031, 13715317, 13721462, 13721472, 13722972, 13724023, 13724171, 13733526, 13733781, 13770612, 13771025, 13774998, 13775246, 13800866, 13807681, 13811740, 13822086, 13834447, 13839731, 13846708, 13847023, 13853003, 13862794, 13872359, 13875470, 13877425, 13882764, 13884451, 13888714, 13888718, 13888720, 13888728, 13888734, 13888742, 13901280, 13951099, 13952785, 13969829, 13969831, 13970234, 13971049, 13987296, 13992756, 13993153, 13993176, 14004059, 14015875, 14015879, 14015881, 14015915, 14015917, 14015942, 14026239, 14033544, 14033547, 14046529, 14046530, 14046559, 14046619, 14046641, 14047279, 14049514, 14049527, 14094557, 14117981, 14123494, 14124094, 14124171, 14128665, 14134756, 14158206, 14173122, 14177552, 14177588, 14177634, 14177859, 14179580, 14185707, 14185812, 14185871, 14189499, 14192584, 14210494, 14254885, 14282921, 14283148, 14283151, 14296718, 14296750, 14296754, 14297199, 14297404, 14311589, 14312743, 14314939, 14324713, 14333850, 14334590, 14357745, 14362897, 14364367, 14377171, 14385482, 14390263, 14411496, 14417866, 14422750, 14435737, 14462940, 14471209, 14480656, 14486954, 14488288, 14493236, 14518535, 14520976, 14522087, 14525099, 14526389, 14526769, 14529574, 14529686, 14529904, 14529910, 14535038, 14535540, 14545339, 14568636, 14568649, 14568656, 14569342, 14570006, 14571560, 14575108, 14575555, 14580151, 14580180, 14580269, 14582446, 14584942, 14585023, 14589068, 14601150, 14602378, 14606696, 14616399, 14622667, 14645740, 14647311, 14653472, 14667107, 14706170, 14711463, 14713557, 14720610, 14726733, 14731375, 14740930, 14749474, 14759620, 14768297, 14781840, 14791395, 14805118, 14805138, 14805146, 14805156, 14805190, 14819297, 14821386, 14827321, 14836029, 14846213, 14846292, 14846347, 14849724, 14854690, 14858734, 14858828, 14859859, 14885020, 14915562, 14915632, 14930678, 14935893, 14938769, 14940592, 14941915, 14943488, 14944163, 14962414, 14962429, 14962448, 14962473, 14962505, 14962551, 14964130, 14964268, 14977178, 15002394, 15007952, 15008619, 15021405, 15062885, 15065511, 15065516, 15083934, 15084936, 15107262, 15108780, 15124337, 15147135, 15236969, 15236972, 15249345, 15269302, 15314506, 15320573, 15320574, 15320583, 15320669, 15321383, 15329691, 15347783, 15356208, 15379496, 15389044, 15392713, 15393285, 15397480, 15400685, 15400694, 15412806, 15413850, 15413908, 15413932, 15414009, 15414176, 15419211, 15421629, 15422075, 15422102, 15469088, 15470499, 15470518, 15470538, 15470541, 15470548, 15470801, 15486562, 15512405, 15527617, 15530664, 15530683, 15530735, 15530775, 15530837, 15533165, 15534610, 15546471, 15553759, 15567582, 15569467, 15569533, 15573745, 15578854, 15580747, 15591466, 15596536, 15628134, 15651714, 15652128, 15652138, 15674969, 15676913, 15688079, 15690052, 15693878, 15712002, 15712095, 15714577, 15741221, 15752245, 15756875, 15776176, 15776184, 15776387, 15821097, 15821296, 15839807, 15843501, 15848699, 15864335, 15896663, 15933218, 16007538, 16008954, 16025904, 16036032, 16036165, 16036191, 16036237, 16036238, 16036240, 16036273, 16036310, 16042538, 16048007, 16053881, 16060450, 16072566, 16072796, 16072974, 16073654, 16075646, 16076286, 16091425, 16100557, 16118915, 16118916, 16118921, 16118925, 16118926, 16118930, 16118933, 16118934, 16118935, 16118939, 16118944, 16118945, 16118946, 16118948, 16118952, 16118953, 16118956, 16118962, 16118967, 16118968, 16118974, 16118975, 16118989, 16119004, 16119041, 16119062, 16119201, 16131777, 16156201, 16169953, 16173941, 16178944, 16198247, 16217109, 16221883, 16224827, 16233818, 16234411, 16234466, 16259538, 16259873, 16259924, 16260265, 16260275, 16260286, 16260683, 16266664, 16274934, 16274943, 16274959, 16274960, 16274971, 16274975, 16274977, 16274980, 16274987, 16275000, 16275004, 16275011, 16275034, 16275037, 16275646, 16275666, 16275712, 16275715, 16276958, 16278661, 16278907, 16279279, 16282795, 16283669, 16286901, 16287017, 16289034, 16292602, 16292806, 16295311, 16295622, 16312805, 16351482, 16363471, 16367906, 16372603, 16378761, 16400748, 16400810, 16400813, 16400816, 16400848, 16401428, 16401605, 16447329, 16459392, 16459755, 16459808, 16467707, 16469834, 16477185, 16477254, 16481886, 16489171, 16490320, 16495612, 16496372, 16512091, 16517649, 16549906, 16552695, 16553957, 16555347, 16555440, 16585362, 16585422, 16603856, 16603909, 16604637, 16604830, 16606122, 16606152, 16606158, 16606177, 16606178, 16606265, 16611874, 16613655, 16615398, 16615408, 16617920, 16621116, 16635060, 16646754, 16660740, 16664751, 16666879, 16666927, 16666929, 16666933, 16666937, 16666939, 16666943, 16666944, 16666951, 16666953, 16666955, 16666958, 16666977, 16666978, 16666979, 16666988, 16666991, 16666992, 16666995, 16666999, 16667001, 16667002, 16667004, 16667012, 16667014, 16667017, 16667019, 16667024, 16667025, 16667026, 16667033, 16667034, 16667036, 16667050, 16667052, 16667061, 16667070, 16667087, 16667109, 16667195, 16675044, 16675215, 16675228, 16679494, 16712341, 16718640, 16722612, 16723014, 16727046, 16727064, 16727068, 16727106, 16727160, 16730340, 16757274, 16757580, 16757858, 16757861, 16757871, 16757873, 16757874, 16757877, 16757879, 16757880, 16757882, 16757884, 16757885, 16757893, 16757894, 16757896, 16757908, 16757909, 16757910, 16757921, 16757926, 16757929, 16757930, 16757933, 16759226, 16765752, 16769397, 16775680, 16783882, 16797529, 16803150, 16803160, 16803164, 16803181, 16803182, 16803184, 16803186, 16803191, 16803193, 16803197, 16803199, 16803205, 16803212, 16803230, 16803235, 16803243, 16803244, 16803271, 16803277, 16803278, 16803280, 16803303, 16803306, 16803315, 16803457, 16803515, 16803979, 16804050, 16804352, 16804355, 16804362, 16804599, 16804748, 16811672, 16815037, 16815949, 16816079, 16845930, 16853875, 16854493, 16855701, 16868730, 16868795, 16929851, 16940477, 16940509, 16942336, 16943326, 16944088, 16946769, 16946796, 16946807, 16949754, 16951211, 16981930, 17013805, 17014129, 17014167, 17014185, 17048612, 17048646, 17081088, 17091494, 17092899, 17107374, 17109957, 17110852, 17126636, 17128039, 17137468, 17151491, 17151492, 17151495, 17151497, 17151504, 17159457, 17161649, 17182204, 17182205, 17185984, 17187509, 17191416, 17191461, 17197102, 17197108, 17197135, 17199697, 17208301, 17214469, 17219923, 17223321, 17226695, 17226762, 17264275, 17264908, 17266950, 17275216, 17275323, 17276635, 17288548, 17312629, 17340053, 17349849, 17354092, 17354697, 17355281, 17357357, 17373941, 17374108, 17374152, 17379029, 17379132, 17381818, 17385311, 17385570, 17419167, 17420551, 17424421, 17426365, 17427623, 17430757, 17445657, 17445788, 17445900, 17447991, 17451904, 17451986, 17452911, 17453760, 17454714, 17465873, 17467045, 17474163, 17478881, 17485269, 17488771, 17494233, 17496695, 17496697, 17496699, 17499319, 17512438, 17512494, 17512563, 17512598, 17513023, 17514208, 17515076, 17516470, 17525909, 17545206, 17552444, 17553206, 17561188, 17582418, 17585119, 17603092, 17603704, 17603714, 17627058, 17628550, 17629767, 17638446, 17649947, 17669440, 17684571, 17732505, 17736438, 17757242, 17771705, 17771720, 17771795, 17771796, 17771840, 17771926, 17772323, 17773040, 17775819, 17781854, 17782504, 17792793, 17794763, 17796998, 17797099, 17797105, 17797380, 17800826, 17872410, 17872424, 17872550, 17872986, 17875092, 17881176, 17890407, 17923741, 17924714, 17944043, 17944392, 17994800, 18005732, 18005785, 18005882, 18010852, 18010895, 18010924, 18011000, 18013060, 18016452, 18033245, 18045123, 18055725, 18077461, 18077675, 18080394, 18097218, 18104374, 18106302, 18113865, 18137563, 18139908, 18174957, 18193822, 18193824, 18193833, 18196928, 18202069, 18202516, 18204299, 18264619, 18292310, 18302904, 18306604, 18316845, 18346617, 18352318, 18357124, 18359252, 18364794, 18364832, 18366222, 18370123, 18370356, 18370375, 18370400, 18370701, 18370744, 18374575, 18380293, 18384325, 18389809, 18389909, 18390358, 18390413, 18390460, 18390470, 18391038, 18396363, 18398788, 18398815, 18398840, 18398867, 18398869, 18398886, 18398899, 18398946, 18398981, 18398991, 18399229, 18402349, 18410224, 18412036, 18423559, 18423602, 18426357, 18426577, 18426727, 18427179, 18432197, 18440315, 18443346, 18450917, 18460192, 18474254, 18475159, 18506714, 18512759, 18513076, 18513756, 18516556, 18525509, 18530237, 18530704, 18530721, 18531553, 18531646, 18531680, 18531750, 18542440, 18544140, 18544498, 18546425, 18550542, 18551500, 18552962, 18557377, 18575753, 18581248, 18586612, 18589157, 18590475, 18595230, 18603232, 18603234, 18603301, 18661549, 18661810, 18673139, 18675681, 18676640, 18676959, 18684874, 18685608, 18704073, 18705811, 18706093, 18707751, 18723803, 18724743, 18725230, 18736597, 18754397, 18761761, 18767037, 18767130, 18767204, 18767354, 18767385, 18767411, 18767412, 18767679, 18767684, 18767689, 18767690, 18767696, 18767697, 18767699, 18767706, 18767709, 18767713, 18767719, 18767737, 18767769, 18767775, 18767776, 18768135, 18768170, 18768312, 18768539, 18771625, 18772635, 18772760, 18774200, 18774934, 18784065, 18785567, 18785790, 18787782, 18787783, 18787784, 18787786, 18787787, 18787790, 18787799, 18787802, 18787804, 18793172, 18801347, 18811795, 18817090, 18822630, 18822660, 18824459, 18824524, 18826070, 18826643, 18826667, 18826689, 18826705, 18826712, 18826728, 18826843, 18826845, 18826846, 18826856, 18826859, 18826889, 18827050, 18827214, 18827645, 18831010, 18833150, 18852343, 18853016, 18861686, 18864053, 18887389, 18887418, 18887443, 18887659, 18913185, 18915158, 18915186, 18930120, 18944000, 18960363, 18980042, 18991334, 19044637, 19053200, 19061791, 19069958, 19071273, 19073582, 19077436, 19089134, 19092842, 19096648, 19098420, 19104321, 19105377, 19105427, 19110052, 19127835, 19137296, 19137478, 19137549, 19137597, 19147065, 19147200, 19148116, 19149702, 19175175, 19176329, 19179497, 19179637, 19180018, 19180067, 19180112, 19186809, 19186860, 19209630, 19212108, 19213292, 19213542, 19214966, 19220661, 19223688, 19228613, 19233417, 19234552, 19257873, 19286534, 19286540, 19297985, 19300294, 19317896, 19349386, 19377811, 19393881, 19432150, 19436530, 19443949, 19464003, 19469208, 19472370, 19495611, 19495612, 19495613, 19495614, 19495615, 19495617, 19495618, 19495621, 19495629, 19495633, 19495640, 19495641, 19495645, 19495668, 19496458, 19497356, 19500138, 19524334, 19530428, 19530924, 19530942, 19531992, 19560008, 19579900, 19593662, 19607487, 19612395, 19612634, 19619691, 19626662, 19655148, 19683389, 19685219, 19685249, 19685253, 19685257, 19688069, 19690219, 19708621, 19821175, 19821181, 19821185, 19821191, 19821192, 19821193, 19821207, 19821208, 19821212, 19821214, 19821223, 19821227, 19821230, 19821276, 19821287, 19825758, 19826311, 19830276, 19860579, 19860593, 19860594, 19860596, 19860602, 19860605, 19860611, 19860612, 19860613, 19860614, 19860615, 19860624, 19860632, 19860652, 19860665, 19862108, 19872386, 19873103, 19873160, 19875772, 19901245, 19932567, 19937154, 19937175, 19937403, 19937505, 19937751, 19937777, 19937778, 19974892, 19975194, 19975396, 19975410, 19975413, 19975423, 19975437, 19991249, 20030302, 20079124, 20102798, 20103300, 20107747, 20112511, 20113428, 20115162, 20115169, 20115192, 20115454, 20115491, 20115608, 20115652, 20124477, 20124812, 20124923, 20125327, 20125348, 20128766, 20129972, 20133880, 20137842, 20141996, 20151258, 20155561, 20161945, 20164162, 20169453, 20172147, 20176054, 20176105, 20176155, 20176277, 20181251, 20204106, 20221396, 20238991, 20239051, 20239412, 20244047, 20252879, 20252882, 20252885, 20253800, 20254453, 20255603, 20256993, 20257043, 20257143, 20298837, 20298927, 20299256, 20299805, 20302512, 20351734, 20352591, 20352592, 20398634, 20431249, 20447244, 20450830, 20468070, 20472833, 20479370, 20483164, 20483866, 20493555, 20497989, 20504122, 20504833, 20504920, 20506421, 20506433, 20510062, 20519204, 20521341, 20529508, 20533122, 20538449, 20545348, 20545517, 20545527, 20545585, 20560720, 20563329, 20563437, 20563573, 20563622, 20563856, 20566296, 20567144, 20571580, 20571910, 20571914, 20576149, 20576753, 20577945, 20584912, 20592302, 20592843, 20596846, 20621519, 20622378, 20622379, 20622380, 20622381, 20622382, 20622383, 20622411, 20622437, 20622465, 20622525, 20623123, 20623138, 20623196, 20623201, 20623237, 20623262, 20623263, 20623301, 20627026, 20627649, 20634891, 20637820, 20637831, 20637838, 20637844, 20637899, 20637922, 20640335, 20643272, 20643998, 20644258, 20644606, 20644941, 20645091, 20648656, 20648679, 20649299, 20654837, 20658077, 20658421, 20664023, 20664131, 20671919, 20672031, 20672040, 20672042, 20672086, 20672143, 20672162, 20691562, 20703641, 20703660, 20703802, 20703825, 20704077, 20716070, 20718938, 20721557, 20724680, 20743378, 20743398, 20743783, 20749791, 20766777, 20781130, 20784536, 20786534, 20831058, 20870139, 20870448, 20870450, 20874920, 20876294, 20879218, 20879782, 20879783, 20879792, 20879820, 20880345, 20918811, 20923019, 20931970, 20932578, 20932579, 20939176, 20946791, 20952863, 20954390, 20954497, 20957047, 20962267, 20965512, 20973468, 20976603, 20978051, 21099696, 21119694, 21119737, 21119824, 21119825, 21119839, 21182920, 21186925, 21229733, 21252219, 21256164, 21277771, 21302656, 21331501, 21337339, 21363124, 21363142, 21363151, 21363156, 21363193, 21363270, 21363313, 21363453, 21363912, 21366183, 21409822, 21473465, 21475231)
            THEN true
            ELSE false
        END as youthmappers
    FROM user_mapping_stats a
    LEFT JOIN changesets_2023 b ON
        a.first_changeset = b.changeset_id 
)
TO 'user_stats_2023.csv' (HEADER, DELIMITER ',');
"""
print(sql)


CREATE TABLE user_mapping_stats AS
SELECT
    user_id,
    count(*) as n_edits,
    min(date_trunc('day', epoch_ms(changeset_timestamp))) as min_timestamp,
    max(date_trunc('day', epoch_ms(changeset_timestamp))) as max_timestamp,
    count(distinct date_trunc('day', epoch_ms(changeset_timestamp))) as n_days,
    min(changeset_id) as first_changeset
FROM read_parquet("/data/processing/partitioned-ohsome-parquet-100997/*/*/*.parquet", hive_partitioning=1)
GROUP BY user_id
HAVING min_timestamp >= '2023-01-01';

CREATE TABLE changesets_2023 AS
SELECT
    changeset_id,
    hashtags
FROM read_parquet("/data/processing/partitioned-ohsome-parquet-100997/*/*/*.parquet", hive_partitioning=1)
WHERE epoch_ms(changeset_timestamp) >= '2023-01-01'
GROUP BY changeset_id, hashtags;
    
COPY (
    SELECT 
        a.*,
        b.hashtags,
        list_has_any(b.hashtags, ['missingmaps', 'MissingMaps', 'missing_maps']) as missing_maps,
        list_contains(list_transform(b.hashtags, x -> x like 'hoto

In [44]:
(27099, 54231, 55024, 82774, 128569, 171231, 178521, 191308, 315995, 366834, 386103, 393098, 393359, 408769, 507262, 561750, 563706, 563947, 593765, 604227, 604538, 643900, 673875, 676762, 918955, 1050248, 1210319, 1217047, 1521733, 1535264, 1639773, 1725054, 1727004, 1769935, 1770239, 1778694, 1779108, 1793105, 1803680, 1803854, 1902578, 1915192, 1935178, 1957873, 1964962, 1998982, 2022154, 2052228, 2111087, 2130214, 2132030, 2150621, 2187425, 2204318, 2245758, 2248202, 2280594, 2327399, 2327410, 2327498, 2327521, 2329822, 2330248, 2331874, 2385561, 2405982, 2589926, 2593307, 2605569, 2623654, 2624064, 2732604, 2757453, 2783807, 2789434, 2789436, 2803704, 2815311, 2815313, 2816546, 2819243, 2824952, 2828866, 2862016, 2869420, 2874300, 2913188, 2913194, 2913504, 2914467, 2947555, 2975705, 3042083, 3044490, 3046993, 3050032, 3112087, 3113029, 3115457, 3116033, 3116122, 3117261, 3117282, 3117626, 3125906, 3127011, 3180585, 3184504, 3236991, 3291762, 3303302, 3338106, 3345135, 3345775, 3349119, 3363190, 3365317, 3367229, 3367241, 3367243, 3367245, 3367248, 3367251, 3367252, 3367313, 3369319, 3376856, 3376862, 3381287, 3381374, 3391305, 3396014, 3397290, 3400042, 3401338, 3407771, 3455921, 3471975, 3481032, 3481073, 3485020, 3502753, 3502761, 3502769, 3502771, 3502824, 3502845, 3504693, 3505733, 3509533, 3512474, 3512495, 3512550, 3512887, 3520331, 3520401, 3520403, 3522458, 3524470, 3536908, 3548727, 3548733, 3551459, 3553162, 3553170, 3553481, 3565118, 3573216, 3575045, 3575055, 3580954, 3582506, 3588829, 3592102, 3592184, 3593219, 3593426, 3596562, 3601934, 3608418, 3609913, 3609914, 3625162, 3627450, 3627707, 3627995, 3628443, 3628506, 3628546, 3628921, 3629560, 3630974, 3632982, 3633498, 3634256, 3636462, 3636468, 3636884, 3639250, 3642288, 3645531, 3648428, 3649667, 3650980, 3660849, 3660854, 3668314, 3671339, 3682432, 3686622, 3686666, 3693601, 3695164, 3734102, 3737095, 3748269, 3767710, 3769220, 3769454, 3769470, 3794884, 3794900, 3796596, 3830109, 3833584, 3851725, 3851730, 3851737, 3851749, 3854561, 3856913, 3859901, 3878185, 3886087, 3888722, 3888755, 3888812, 3888908, 3889552, 3930537, 3934069, 3951928, 3975528, 3996686, 4010972, 4011403, 4015712, 4023437, 4039113, 4063492, 4110434, 4149748, 4202290, 4223917, 4316098, 4319101, 4352853, 4366372, 4366421, 4366422, 4366434, 4366436, 4379874, 4419442, 4434231, 4434270, 4456266, 4456639, 4456643, 4456655, 4456657, 4456665, 4456732, 4458831, 4459284, 4459720, 4459793, 4466190, 4466195, 4466209, 4467310, 4470074, 4470083, 4471622, 4471623, 4471624, 4471625, 4471628, 4471629, 4471643, 4471655, 4471851, 4476246, 4479975, 4516636, 4516652, 4516666, 4516681, 4516684, 4516743, 4516804, 4516830, 4517943, 4518610, 4518760, 4521382, 4530401, 4530956, 4530972, 4530980, 4531021, 4531045, 4531059, 4531060, 4531377, 4533410, 4541953, 4542220, 4543032, 4543231, 4543255, 4543524, 4543600, 4543850, 4543864, 4543868, 4545114, 4545134, 4545135, 4545193, 4545213, 4545231, 4545678, 4545852, 4550348, 4550903, 4550905, 4551656, 4551660, 4551663, 4551664, 4551682, 4551705, 4551931, 4556176, 4563312, 4563883, 4564910, 4564929, 4569905, 4576568, 4577601, 4582659, 4585394, 4587793, 4587800, 4589870, 4610319, 4610340, 4610363, 4610468, 4610568, 4610711, 4610725, 4621166, 4621346, 4621352, 4630994, 4631420, 4645012, 4645947, 4653136, 4653441, 4653773, 4655090, 4655091, 4655093, 4655096, 4655115, 4655120, 4672373, 4672528, 4681094, 4685020, 4690780, 4694285, 4714195, 4719316, 4723292, 4723377, 4737801, 4742359, 4750403, 4750411, 4750414, 4750466, 4750581, 4755369, 4755418, 4755435, 4755754, 4755768, 4755790, 4780240, 4795315, 4795320, 4795323, 4795332, 4795340, 4795347, 4795375, 4795381, 4795397, 4795416, 4795420, 4795500, 4795506, 4797131, 4797156, 4797177, 4797229, 4804704, 4810380, 4810482, 4810484, 4810485, 4814747, 4814822, 4814987, 4816007, 4816114, 4825148, 4825748, 4826501, 4828025, 4829257, 4831172, 4831302, 4834302, 4835663, 4836584, 4837357, 4837361, 4837426, 4838813, 4838903, 4846853, 4849501, 4853368, 4853369, 4853701, 4853783, 4853807, 4853808, 4853836, 4853839, 4853859, 4853867, 4854138, 4854623, 4854783, 4854861, 4854939, 4857822, 4858154, 4858652, 4860449, 4861141, 4861563, 4861595, 4861681, 4861695, 4861938, 4862041, 4862042, 4862063, 4862072, 4862228, 4863679, 4863686, 4863699, 4863848, 4873392, 4873427, 4873431, 4873450, 4873470, 4874033, 4874102, 4887075, 4899670, 4908508, 4909982, 4928257, 4950300, 4971333, 4977551, 4978827, 5010433, 5022087, 5037631, 5039176, 5093311, 5112404, 5117109, 5129203, 5136132, 5136182, 5141416, 5147024, 5149809, 5153612, 5154063, 5179309, 5188606, 5191095, 5192895, 5193350, 5195345, 5195536, 5198564, 5209776, 5209801, 5209819, 5209824, 5212877, 5227090, 5236091, 5236947, 5239509, 5240907, 5254921, 5257863, 5266174, 5267631, 5270698, 5275155, 5275165, 5275678, 5278057, 5281513, 5290000, 5291232, 5294434, 5294450, 5295021, 5298447, 5298569, 5299988, 5307929, 5310527, 5310986, 5316526, 5319796, 5319799, 5319805, 5319814, 5319817, 5320010, 5320013, 5320021, 5320024, 5320249, 5320264, 5320413, 5323277, 5323284, 5323285, 5323296, 5323301, 5323310, 5323336, 5326054, 5326154, 5328123, 5329058, 5330232, 5330990, 5331092, 5331204, 5331225, 5331297, 5331690, 5332045, 5337025, 5337097, 5337261, 5340340, 5340528, 5365592, 5374523, 5377023, 5379313, 5379337, 5392008, 5421563, 5422174, 5444270, 5448960, 5449028, 5455971, 5466476, 5466597, 5467159, 5484092, 5484094, 5484300, 5484336, 5484541, 5489383, 5545936, 5562404, 5578399, 5585721, 5596182, 5615669, 5615898, 5624644, 5624958, 5625355, 5627718, 5629648, 5646866, 5655451, 5658513, 5659317, 5659378, 5659715, 5689003, 5696137, 5696198, 5702683, 5707143, 5708406, 5718341, 5726905, 5726933, 5748265, 5748734, 5752250, 5752629, 5854692, 5863675, 5867004, 5867434, 5867704, 5881052, 5891083, 5897965, 5921856, 5942196, 5942517, 5942719, 5942862, 5943155, 5943289, 5943826, 6000145, 6000163, 6000171, 6000181, 6000183, 6000211, 6000380, 6001174, 6001440, 6017386, 6058846, 6067659, 6067668, 6074987, 6075008, 6075030, 6075038, 6075041, 6075053, 6075257, 6106947, 6111679, 6125882, 6126808, 6136314, 6150323, 6163141, 6163623, 6173770, 6175560, 6175966, 6176290, 6177995, 6179768, 6192228, 6192530, 6192851, 6195345, 6200023, 6201821, 6201856, 6201866, 6202367, 6202387, 6204667, 6206544, 6207019, 6211207, 6212630, 6212762, 6212799, 6213019, 6213371, 6233686, 6234449, 6237421, 6239474, 6239511, 6239541, 6239577, 6239855, 6241856, 6242902, 6243439, 6246337, 6249131, 6249877, 6249886, 6249908, 6249923, 6306737, 6307214, 6309404, 6312144, 6312212, 6313846, 6317343, 6322650, 6323155, 6323172, 6323223, 6323411, 6329771, 6334327, 6337421, 6342184, 6342189, 6342214, 6342278, 6352785, 6359620, 6362084, 6405430, 6406545, 6411973, 6415223, 6465366, 6465381, 6465429, 6465623, 6465640, 6465660, 6465671, 6465688, 6465700, 6465713, 6465978, 6466697, 6474298, 6475519, 6481304, 6503849, 6503859, 6503872, 6507231, 6509614, 6509998, 6520163, 6520256, 6520301, 6528197, 6528213, 6528235, 6529513, 6529630, 6529746, 6529792, 6530414, 6533575, 6534235, 6535064, 6535648, 6535722, 6535879, 6539988, 6540873, 6545197, 6546372, 6546934, 6547685, 6549191, 6549194, 6552110, 6555839, 6556202, 6557104, 6557317, 6557448, 6557742, 6559755, 6599895, 6606782, 6636249, 6636922, 6638443, 6643932, 6648710, 6652088, 6652179, 6653283, 6660054, 6660094, 6660119, 6660144, 6660182, 6660191, 6661941, 6665480, 6674701, 6675564, 6677927, 6680704, 6680715, 6681057, 6688997, 6689154, 6689453, 6689456, 6689921, 6689931, 6696681, 6696709, 6711663, 6713759, 6715932, 6720602, 6723048, 6726590, 6726596, 6726600, 6726607, 6726616, 6726619, 6726623, 6726647, 6726661, 6726672, 6726677, 6726686, 6726693, 6726702, 6726711, 6726737, 6726738, 6733614, 6734546, 6746561, 6753040, 6755994, 6791097, 6791625, 6793349, 6793356, 6795786, 6795789, 6797474, 6797922, 6802216, 6806919, 6812135, 6812140, 6812144, 6812353, 6819172, 6829138, 6831676, 6831973, 6841161, 6841213, 6842923, 6866977, 6873995, 6882856, 6891510, 6893434, 6893771, 6893935, 6895044, 6895483, 6903002, 6903073, 6903135, 6935059, 6935263, 6950137, 6962560, 6962787, 6962791, 6962796, 6962813, 6962827, 6962837, 6967037, 6968212, 6968661, 6980380, 6980804, 6980806, 6980809, 6980813, 6980816, 6980818, 6980820, 6980821, 6980824, 6981028, 6981031, 6981033, 6981036, 6981041, 6981046, 6981047, 6981048, 6981052, 6981053, 6981056, 6982496, 6984710, 6987266, 6989772, 6989797, 6989799, 6989803, 6989928, 6989983, 6990048, 6990114, 6990860, 6990887, 6991194, 6991874, 6991886, 6991897, 6992012, 6992840, 6994816, 6997855, 6997856, 6997863, 6997866, 6997867, 6997950, 6997999, 6998065, 6998399, 6998858, 7001461, 7001522, 7001528, 7001606, 7002112, 7009028, 7009378, 7009393, 7009482, 7009574, 7009576, 7009578, 7009582, 7009584, 7009585, 7009586, 7009587, 7009588, 7009590, 7009592, 7009594, 7009595, 7009597, 7009603, 7009657, 7009794, 7010032, 7034610, 7053510, 7063886, 7115639, 7115972, 7124841, 7131883, 7154329, 7159298, 7162262, 7164287, 7169521, 7179226, 7179671, 7214903, 7217628, 7233400, 7242430, 7242581, 7243359, 7243426, 7243605, 7254945, 7255043, 7255659, 7354300, 7410158, 7447450, 7516714, 7529952, 7533975, 7567509, 7599517, 7600327, 7600329, 7606880, 7622589, 7624135, 7631742, 7633451, 7657619, 7658859, 7666560, 7666645, 7666726, 7681679, 7689102, 7689176, 7692594, 7692596, 7692631, 7692654, 7692656, 7692663, 7692664, 7692666, 7692679, 7692681, 7692684, 7692688, 7692693, 7692695, 7692969, 7703465, 7708747, 7709395, 7710394, 7713992, 7714106, 7734979, 7777526, 7795270, 7823217, 7834299, 7858115, 7858616, 7911952, 7911964, 7912018, 7912243, 7912251, 7912256, 7912257, 7912263, 7912298, 7912312, 7912315, 7949960, 7950417, 7964630, 7964812, 7966705, 7977256, 8102301, 8102312, 8110709, 8138348, 8155627, 8220038, 8233250, 8253524, 8277193, 8288535, 8288596, 8292606, 8292630, 8292649, 8292657, 8292727, 8293676, 8297568, 8309360, 8310862, 8311514, 8315456, 8316170, 8317296, 8317902, 8330489, 8333271, 8353419, 8377453, 8380708, 8386618, 8389251, 8392175, 8413832, 8414406, 8425742, 8431311, 8472066, 8474278, 8474299, 8474365, 8474575, 8478755, 8484471, 8502322, 8514452, 8521341, 8526180, 8535118, 8535574, 8543379, 8553006, 8555267, 8557611, 8557815, 8564211, 8564467, 8576693, 8593936, 8609412, 8626264, 8640661, 8640714, 8640750, 8668411, 8683526, 8684339, 8693193, 8698377, 8733281, 8733294, 8734324, 8734332, 8738512, 8747689, 8749272, 8772532, 8773807, 8773991, 8774532, 8801128, 8809024, 8822271, 8825311, 8825695, 8825750, 8827882, 8833032, 8836427, 8852922, 8852953, 8853135, 8857309, 8857496, 8871263, 8871486, 8871772, 8874443, 8879119, 8880467, 8881365, 8882421, 8883831, 8884106, 8884110, 8884245, 8885047, 8885056, 8885127, 8885135, 8885248, 8886943, 8887664, 8889983, 8890000, 8899303, 8930627, 8934031, 8971246, 8991600, 8992397, 9016554, 9019904, 9021289, 9028962, 9028970, 9039501, 9054148, 9059201, 9059245, 9060686, 9060689, 9060690, 9060692, 9063710, 9063754, 9064751, 9067142, 9067419, 9090183, 9093600, 9094296, 9094298, 9094303, 9094312, 9094337, 9097672, 9097678, 9097681, 9097700, 9097882, 9098075, 9125001, 9155658, 9160253, 9163481, 9167598, 9168467, 9202190, 9215960, 9223721, 9232250, 9234461, 9264274, 9264317, 9272311, 9342997, 9351084, 9356680, 9361895, 9362957, 9374154, 9383528, 9389700, 9394713, 9398759, 9398793, 9398865, 9402325, 9426620, 9426669, 9436646, 9446263, 9446269, 9446286, 9449447, 9449463, 9459426, 9462170, 9463670, 9467533, 9476543, 9476545, 9476547, 9486708, 9489244, 9489250, 9489296, 9489407, 9491276, 9492821, 9513771, 9513779, 9513805, 9522913, 9525499, 9530563, 9531596, 9533675, 9534424, 9544112, 9553534, 9553539, 9554224, 9555656, 9557083, 9570396, 9571250, 9572693, 9574037, 9574249, 9574432, 9574619, 9574624, 9574627, 9574629, 9574638, 9580890, 9580891, 9580898, 9580960, 9580964, 9602004, 9602857, 9619006, 9620670, 9622928, 9624650, 9624795, 9625064, 9636868, 9636881, 9640365, 9648272, 9926812, 9946255, 9946686, 9946801, 9964969, 9966136, 9966880, 9969873, 10000054, 10002113, 10019356, 10019754, 10019802, 10020481, 10020547, 10020576, 10021541, 10021808, 10030205, 10033761, 10044753, 10045809, 10047805, 10047837, 10062444, 10068373, 10068412, 10069125, 10071053, 10072236, 10073390, 10083341, 10110280, 10111729, 10118836, 10125196, 10126896, 10128086, 10137724, 10137775, 10138066, 10138238, 10138560, 10138757, 10138825, 10139406, 10139430, 10139623, 10140159, 10140184, 10140186, 10140227, 10140358, 10140402, 10140572, 10141068, 10141120, 10142645, 10150885, 10162073, 10162076, 10162079, 10162080, 10162083, 10162087, 10162089, 10162096, 10162097, 10162099, 10162100, 10162105, 10162109, 10162110, 10162111, 10162113, 10162120, 10162122, 10162536, 10163273, 10170056, 10174407, 10184312, 10186120, 10194382, 10194383, 10194384, 10194386, 10194387, 10194388, 10194389, 10194390, 10194391, 10194392, 10194393, 10194394, 10194398, 10194415, 10194430, 10196983, 10197080, 10205518, 10211495, 10218760, 10227210, 10228236, 10242484, 10245957, 10246198, 10249523, 10252701, 10252703, 10252708, 10252710, 10252714, 10252716, 10252720, 10252728, 10252731, 10252732, 10252737, 10252739, 10252752, 10252757, 10252761, 10252762, 10252792, 10253135, 10253243, 10254841, 10257494, 10257798, 10257929, 10264522, 10271734, 10279009, 10285872, 10294659, 10295058, 10296067, 10296332, 10297931, 10298376, 10298452, 10298667, 10299297, 10300870, 10300871, 10300875, 10300876, 10300877, 10300880, 10300881, 10300882, 10300888, 10300893, 10300895, 10300899, 10300907, 10300919, 10300995, 10301056, 10303316, 10303893, 10303940, 10305086, 10305121, 10306905, 10307099, 10307182, 10307196, 10307406, 10307836, 10307856, 10307860, 10307869, 10309225, 10309398, 10309693, 10310242, 10310563, 10310602, 10310605, 10310739, 10310781, 10312497, 10313455, 10314035, 10314775, 10314916, 10315213, 10315233, 10315245, 10315246, 10315286, 10315810, 10315817, 10315823, 10315824, 10315827, 10315844, 10315855, 10315899, 10315904, 10315915, 10316037, 10316295, 10317067, 10324752, 10325675, 10328649, 10332104, 10333952, 10334118, 10335385, 10335471, 10336280, 10344444, 10344453, 10348336, 10350923, 10350932, 10352738, 10352796, 10352870, 10353849, 10353875, 10353905, 10353909, 10353911, 10365273, 10365640, 10365739, 10366307, 10369293, 10371572, 10371825, 10375489, 10376870, 10377118, 10377934, 10380110, 10381183, 10381887, 10382376, 10382752, 10382754, 10389211, 10392282, 10392792, 10393901, 10393966, 10394070, 10394108, 10394138, 10394161, 10394186, 10394201, 10394700, 10394713, 10394714, 10394721, 10394726, 10394728, 10394732, 10394737, 10394772, 10394778, 10397517, 10397538, 10397553, 10401038, 10403792, 10408926, 10412037, 10412054, 10413303, 10413514, 10414983, 10414984, 10415004, 10415005, 10415006, 10415007, 10415008, 10415028, 10419753, 10422352, 10422389, 10422391, 10428097, 10428171, 10428195, 10428548, 10428870, 10429396, 10429461, 10431373, 10434543, 10444596, 10447784, 10447791, 10452562, 10453340, 10454980, 10459757, 10461053, 10465311, 10466134, 10466608, 10466749, 10466757, 10466768, 10469823, 10471556, 10472439, 10472441, 10472447, 10472550, 10473429, 10473724, 10476531, 10482675, 10483389, 10483390, 10483398, 10483411, 10483647, 10488582, 10491954, 10517772, 10553167, 10553190, 10569841, 10570996, 10572167, 10572533, 10575987, 10602213, 10602238, 10613104, 10613118, 10613138, 10618943, 10618978, 10618979, 10619016, 10641000, 10660787, 10665962, 10674298, 10683886, 10688519, 10697587, 10701038, 10723548, 10723557, 10723561, 10723563, 10723574, 10723577, 10723578, 10723697, 10734833, 10755763, 10766953, 10766954, 10766956, 10766962, 10766964, 10766969, 10766972, 10766973, 10766983, 10766991, 10766996, 10767006, 10767041, 10767042, 10767048, 10767067, 10768004, 10771699, 10771854, 10773950, 10774428, 10779819, 10784634, 10798401, 10805123, 10812819, 10812904, 10836586, 10837586, 10837709, 10845620, 10847467, 10855988, 10856000, 10856005, 10856028, 10856839, 10858729, 10864467, 10866497, 10866499, 10866500, 10866506, 10866571, 10866643, 10868392, 10868456, 10868648, 10876087, 10876133, 10882647, 10885730, 10885796, 10887171, 10894654, 10898257, 10921442, 10921645, 10934301, 10939841, 10963925, 10978697, 10979233, 10982439, 10984892, 10991352, 10997928, 11010087, 11010237, 11058007, 11084219, 11086590, 11137096, 11141773, 11158572, 11171572, 11171574, 11171592, 11171620, 11177499, 11178862, 11180925, 11295328, 11295416, 11297323, 11385528, 11395973, 11396063, 11396103, 11396143, 11432684, 11446724, 11491044, 11491048, 11491063, 11491064, 11491069, 11491073, 11491077, 11491080, 11491829, 11494405, 11512481, 11536988, 11539361, 11539365, 11539367, 11539368, 11539375, 11550159, 11566011, 11570880, 11572008, 11575121, 11585555, 11589556, 11589559, 11589561, 11589667, 11589867, 11600833, 11603914, 11632336, 11640369, 11640807, 11656047, 11656135, 11674896, 11730586, 11733358, 11774025, 11774060, 11779623, 11793207, 11807790, 11830680, 11835896, 11867310, 11868369, 11900841, 11908540, 11919491, 11943440, 11953893, 11976394, 11999420, 12015769, 12031941, 12067506, 12067647, 12072417, 12078651, 12088780, 12106999, 12109557, 12123439, 12129925, 12144247, 12151924, 12157812, 12176702, 12181999, 12184321, 12191928, 12193439, 12208589, 12208592, 12208593, 12208597, 12208603, 12208608, 12208610, 12208627, 12208662, 12209174, 12209177, 12209189, 12209195, 12209201, 12209216, 12209227, 12209478, 12210147, 12210356, 12210359, 12210417, 12210421, 12210501, 12210569, 12212203, 12212329, 12213080, 12213821, 12214057, 12214092, 12214577, 12214698, 12214787, 12215080, 12216668, 12231845, 12248057, 12248058, 12258107, 12263608, 12277006, 12280809, 12291129, 12311762, 12324205, 12339121, 12342156, 12357640, 12368208, 12378137, 12384423, 12384443, 12384464, 12384592, 12386161, 12387301, 12397323, 12400048, 12406019, 12406042, 12406070, 12406080, 12406088, 12407843, 12408162, 12424765, 12439450, 12441709, 12454484, 12480255, 12484304, 12520176, 12529516, 12531530, 12533960, 12534004, 12545625, 12547895, 12567517, 12572606, 12572714, 12572819, 12575202, 12595407, 12599925, 12615464, 12615466, 12615482, 12620938, 12629068, 12632505, 12640881, 12650130, 12651631, 12666546, 12691149, 12694216, 12696088, 12696819, 12698223, 12748853, 12774177, 12791691, 12796222, 12797808, 12798188, 12800129, 12810081, 12810545, 12810570, 12810572, 12822692, 12827111, 12829874, 12831597, 12847852, 12853146, 12853242, 12853574, 12860486, 12865158, 12865348, 12867442, 12867476, 12867509, 12879973, 12880236, 12880237, 12883224, 12883277, 12883311, 12908085, 12911941, 12918215, 12970584, 12993219, 13000502, 13005097, 13005199, 13005253, 13007399, 13013984, 13014934, 13015130, 13022853, 13023273, 13023274, 13023382, 13031108, 13031156, 13031777, 13039530, 13063544, 13063701, 13067323, 13076013, 13083632, 13086886, 13095983, 13095994, 13110187, 13114924, 13115486, 13117996, 13119139, 13119640, 13119957, 13120060, 13120394, 13123824, 13125074, 13125190, 13125269, 13125293, 13125421, 13140436, 13149325, 13158337, 13158452, 13158530, 13158654, 13158927, 13158938, 13159018, 13159052, 13159097, 13159413, 13162213, 13162222, 13163024, 13163038, 13163041, 13163043, 13163061, 13163078, 13163089, 13163096, 13163111, 13163116, 13163249, 13163363, 13164975, 13164985, 13165036, 13165051, 13165456, 13165468, 13165920, 13166458, 13170728, 13172373, 13180237, 13184051, 13203366, 13205258, 13205671, 13206115, 13207766, 13213172, 13214115, 13246372, 13248604, 13252977, 13253207, 13253583, 13255729, 13255920, 13261176, 13301985, 13322443, 13330486, 13352087, 13369867, 13377221, 13395522, 13401384, 13426455, 13426465, 13426470, 13426490, 13426512, 13442069, 13442272, 13442281, 13442487, 13443059, 13481812, 13529725, 13594329, 13594657, 13600653, 13639411, 13645764, 13685990, 13703031, 13715317, 13721462, 13721472, 13722972, 13724023, 13724171, 13733526, 13733781, 13770612, 13771025, 13774998, 13775246, 13800866, 13807681, 13811740, 13822086, 13834447, 13839731, 13846708, 13847023, 13853003, 13862794, 13872359, 13875470, 13877425, 13882764, 13884451, 13888714, 13888718, 13888720, 13888728, 13888734, 13888742, 13901280, 13951099, 13952785, 13969829, 13969831, 13970234, 13971049, 13987296, 13992756, 13993153, 13993176, 14004059, 14015875, 14015879, 14015881, 14015915, 14015917, 14015942, 14026239, 14033544, 14033547, 14046529, 14046530, 14046559, 14046619, 14046641, 14047279, 14049514, 14049527, 14094557, 14117981, 14123494, 14124094, 14124171, 14128665, 14134756, 14158206, 14173122, 14177552, 14177588, 14177634, 14177859, 14179580, 14185707, 14185812, 14185871, 14189499, 14192584, 14210494, 14254885, 14282921, 14283148, 14283151, 14296718, 14296750, 14296754, 14297199, 14297404, 14311589, 14312743, 14314939, 14324713, 14333850, 14334590, 14357745, 14362897, 14364367, 14377171, 14385482, 14390263, 14411496, 14417866, 14422750, 14435737, 14462940, 14471209, 14480656, 14486954, 14488288, 14493236, 14518535, 14520976, 14522087, 14525099, 14526389, 14526769, 14529574, 14529686, 14529904, 14529910, 14535038, 14535540, 14545339, 14568636, 14568649, 14568656, 14569342, 14570006, 14571560, 14575108, 14575555, 14580151, 14580180, 14580269, 14582446, 14584942, 14585023, 14589068, 14601150, 14602378, 14606696, 14616399, 14622667, 14645740, 14647311, 14653472, 14667107, 14706170, 14711463, 14713557, 14720610, 14726733, 14731375, 14740930, 14749474, 14759620, 14768297, 14781840, 14791395, 14805118, 14805138, 14805146, 14805156, 14805190, 14819297, 14821386, 14827321, 14836029, 14846213, 14846292, 14846347, 14849724, 14854690, 14858734, 14858828, 14859859, 14885020, 14915562, 14915632, 14930678, 14935893, 14938769, 14940592, 14941915, 14943488, 14944163, 14962414, 14962429, 14962448, 14962473, 14962505, 14962551, 14964130, 14964268, 14977178, 15002394, 15007952, 15008619, 15021405, 15062885, 15065511, 15065516, 15083934, 15084936, 15107262, 15108780, 15124337, 15147135, 15236969, 15236972, 15249345, 15269302, 15314506, 15320573, 15320574, 15320583, 15320669, 15321383, 15329691, 15347783, 15356208, 15379496, 15389044, 15392713, 15393285, 15397480, 15400685, 15400694, 15412806, 15413850, 15413908, 15413932, 15414009, 15414176, 15419211, 15421629, 15422075, 15422102, 15469088, 15470499, 15470518, 15470538, 15470541, 15470548, 15470801, 15486562, 15512405, 15527617, 15530664, 15530683, 15530735, 15530775, 15530837, 15533165, 15534610, 15546471, 15553759, 15567582, 15569467, 15569533, 15573745, 15578854, 15580747, 15591466, 15596536, 15628134, 15651714, 15652128, 15652138, 15674969, 15676913, 15688079, 15690052, 15693878, 15712002, 15712095, 15714577, 15741221, 15752245, 15756875, 15776176, 15776184, 15776387, 15821097, 15821296, 15839807, 15843501, 15848699, 15864335, 15896663, 15933218, 16007538, 16008954, 16025904, 16036032, 16036165, 16036191, 16036237, 16036238, 16036240, 16036273, 16036310, 16042538, 16048007, 16053881, 16060450, 16072566, 16072796, 16072974, 16073654, 16075646, 16076286, 16091425, 16100557, 16118915, 16118916, 16118921, 16118925, 16118926, 16118930, 16118933, 16118934, 16118935, 16118939, 16118944, 16118945, 16118946, 16118948, 16118952, 16118953, 16118956, 16118962, 16118967, 16118968, 16118974, 16118975, 16118989, 16119004, 16119041, 16119062, 16119201, 16131777, 16156201, 16169953, 16173941, 16178944, 16198247, 16217109, 16221883, 16224827, 16233818, 16234411, 16234466, 16259538, 16259873, 16259924, 16260265, 16260275, 16260286, 16260683, 16266664, 16274934, 16274943, 16274959, 16274960, 16274971, 16274975, 16274977, 16274980, 16274987, 16275000, 16275004, 16275011, 16275034, 16275037, 16275646, 16275666, 16275712, 16275715, 16276958, 16278661, 16278907, 16279279, 16282795, 16283669, 16286901, 16287017, 16289034, 16292602, 16292806, 16295311, 16295622, 16312805, 16351482, 16363471, 16367906, 16372603, 16378761, 16400748, 16400810, 16400813, 16400816, 16400848, 16401428, 16401605, 16447329, 16459392, 16459755, 16459808, 16467707, 16469834, 16477185, 16477254, 16481886, 16489171, 16490320, 16495612, 16496372, 16512091, 16517649, 16549906, 16552695, 16553957, 16555347, 16555440, 16585362, 16585422, 16603856, 16603909, 16604637, 16604830, 16606122, 16606152, 16606158, 16606177, 16606178, 16606265, 16611874, 16613655, 16615398, 16615408, 16617920, 16621116, 16635060, 16646754, 16660740, 16664751, 16666879, 16666927, 16666929, 16666933, 16666937, 16666939, 16666943, 16666944, 16666951, 16666953, 16666955, 16666958, 16666977, 16666978, 16666979, 16666988, 16666991, 16666992, 16666995, 16666999, 16667001, 16667002, 16667004, 16667012, 16667014, 16667017, 16667019, 16667024, 16667025, 16667026, 16667033, 16667034, 16667036, 16667050, 16667052, 16667061, 16667070, 16667087, 16667109, 16667195, 16675044, 16675215, 16675228, 16679494, 16712341, 16718640, 16722612, 16723014, 16727046, 16727064, 16727068, 16727106, 16727160, 16730340, 16757274, 16757580, 16757858, 16757861, 16757871, 16757873, 16757874, 16757877, 16757879, 16757880, 16757882, 16757884, 16757885, 16757893, 16757894, 16757896, 16757908, 16757909, 16757910, 16757921, 16757926, 16757929, 16757930, 16757933, 16759226, 16765752, 16769397, 16775680, 16783882, 16797529, 16803150, 16803160, 16803164, 16803181, 16803182, 16803184, 16803186, 16803191, 16803193, 16803197, 16803199, 16803205, 16803212, 16803230, 16803235, 16803243, 16803244, 16803271, 16803277, 16803278, 16803280, 16803303, 16803306, 16803315, 16803457, 16803515, 16803979, 16804050, 16804352, 16804355, 16804362, 16804599, 16804748, 16811672, 16815037, 16815949, 16816079, 16845930, 16853875, 16854493, 16855701, 16868730, 16868795, 16929851, 16940477, 16940509, 16942336, 16943326, 16944088, 16946769, 16946796, 16946807, 16949754, 16951211, 16981930, 17013805, 17014129, 17014167, 17014185, 17048612, 17048646, 17081088, 17091494, 17092899, 17107374, 17109957, 17110852, 17126636, 17128039, 17137468, 17151491, 17151492, 17151495, 17151497, 17151504, 17159457, 17161649, 17182204, 17182205, 17185984, 17187509, 17191416, 17191461, 17197102, 17197108, 17197135, 17199697, 17208301, 17214469, 17219923, 17223321, 17226695, 17226762, 17264275, 17264908, 17266950, 17275216, 17275323, 17276635, 17288548, 17312629, 17340053, 17349849, 17354092, 17354697, 17355281, 17357357, 17373941, 17374108, 17374152, 17379029, 17379132, 17381818, 17385311, 17385570, 17419167, 17420551, 17424421, 17426365, 17427623, 17430757, 17445657, 17445788, 17445900, 17447991, 17451904, 17451986, 17452911, 17453760, 17454714, 17465873, 17467045, 17474163, 17478881, 17485269, 17488771, 17494233, 17496695, 17496697, 17496699, 17499319, 17512438, 17512494, 17512563, 17512598, 17513023, 17514208, 17515076, 17516470, 17525909, 17545206, 17552444, 17553206, 17561188, 17582418, 17585119, 17603092, 17603704, 17603714, 17627058, 17628550, 17629767, 17638446, 17649947, 17669440, 17684571, 17732505, 17736438, 17757242, 17771705, 17771720, 17771795, 17771796, 17771840, 17771926, 17772323, 17773040, 17775819, 17781854, 17782504, 17792793, 17794763, 17796998, 17797099, 17797105, 17797380, 17800826, 17872410, 17872424, 17872550, 17872986, 17875092, 17881176, 17890407, 17923741, 17924714, 17944043, 17944392, 17994800, 18005732, 18005785, 18005882, 18010852, 18010895, 18010924, 18011000, 18013060, 18016452, 18033245, 18045123, 18055725, 18077461, 18077675, 18080394, 18097218, 18104374, 18106302, 18113865, 18137563, 18139908, 18174957, 18193822, 18193824, 18193833, 18196928, 18202069, 18202516, 18204299, 18264619, 18292310, 18302904, 18306604, 18316845, 18346617, 18352318, 18357124, 18359252, 18364794, 18364832, 18366222, 18370123, 18370356, 18370375, 18370400, 18370701, 18370744, 18374575, 18380293, 18384325, 18389809, 18389909, 18390358, 18390413, 18390460, 18390470, 18391038, 18396363, 18398788, 18398815, 18398840, 18398867, 18398869, 18398886, 18398899, 18398946, 18398981, 18398991, 18399229, 18402349, 18410224, 18412036, 18423559, 18423602, 18426357, 18426577, 18426727, 18427179, 18432197, 18440315, 18443346, 18450917, 18460192, 18474254, 18475159, 18506714, 18512759, 18513076, 18513756, 18516556, 18525509, 18530237, 18530704, 18530721, 18531553, 18531646, 18531680, 18531750, 18542440, 18544140, 18544498, 18546425, 18550542, 18551500, 18552962, 18557377, 18575753, 18581248, 18586612, 18589157, 18590475, 18595230, 18603232, 18603234, 18603301, 18661549, 18661810, 18673139, 18675681, 18676640, 18676959, 18684874, 18685608, 18704073, 18705811, 18706093, 18707751, 18723803, 18724743, 18725230, 18736597, 18754397, 18761761, 18767037, 18767130, 18767204, 18767354, 18767385, 18767411, 18767412, 18767679, 18767684, 18767689, 18767690, 18767696, 18767697, 18767699, 18767706, 18767709, 18767713, 18767719, 18767737, 18767769, 18767775, 18767776, 18768135, 18768170, 18768312, 18768539, 18771625, 18772635, 18772760, 18774200, 18774934, 18784065, 18785567, 18785790, 18787782, 18787783, 18787784, 18787786, 18787787, 18787790, 18787799, 18787802, 18787804, 18793172, 18801347, 18811795, 18817090, 18822630, 18822660, 18824459, 18824524, 18826070, 18826643, 18826667, 18826689, 18826705, 18826712, 18826728, 18826843, 18826845, 18826846, 18826856, 18826859, 18826889, 18827050, 18827214, 18827645, 18831010, 18833150, 18852343, 18853016, 18861686, 18864053, 18887389, 18887418, 18887443, 18887659, 18913185, 18915158, 18915186, 18930120, 18944000, 18960363, 18980042, 18991334, 19044637, 19053200, 19061791, 19069958, 19071273, 19073582, 19077436, 19089134, 19092842, 19096648, 19098420, 19104321, 19105377, 19105427, 19110052, 19127835, 19137296, 19137478, 19137549, 19137597, 19147065, 19147200, 19148116, 19149702, 19175175, 19176329, 19179497, 19179637, 19180018, 19180067, 19180112, 19186809, 19186860, 19209630, 19212108, 19213292, 19213542, 19214966, 19220661, 19223688, 19228613, 19233417, 19234552, 19257873, 19286534, 19286540, 19297985, 19300294, 19317896, 19349386, 19377811, 19393881, 19432150, 19436530, 19443949, 19464003, 19469208, 19472370, 19495611, 19495612, 19495613, 19495614, 19495615, 19495617, 19495618, 19495621, 19495629, 19495633, 19495640, 19495641, 19495645, 19495668, 19496458, 19497356, 19500138, 19524334, 19530428, 19530924, 19530942, 19531992, 19560008, 19579900, 19593662, 19607487, 19612395, 19612634, 19619691, 19626662, 19655148, 19683389, 19685219, 19685249, 19685253, 19685257, 19688069, 19690219, 19708621, 19821175, 19821181, 19821185, 19821191, 19821192, 19821193, 19821207, 19821208, 19821212, 19821214, 19821223, 19821227, 19821230, 19821276, 19821287, 19825758, 19826311, 19830276, 19860579, 19860593, 19860594, 19860596, 19860602, 19860605, 19860611, 19860612, 19860613, 19860614, 19860615, 19860624, 19860632, 19860652, 19860665, 19862108, 19872386, 19873103, 19873160, 19875772, 19901245, 19932567, 19937154, 19937175, 19937403, 19937505, 19937751, 19937777, 19937778, 19974892, 19975194, 19975396, 19975410, 19975413, 19975423, 19975437, 19991249, 20030302, 20079124, 20102798, 20103300, 20107747, 20112511, 20113428, 20115162, 20115169, 20115192, 20115454, 20115491, 20115608, 20115652, 20124477, 20124812, 20124923, 20125327, 20125348, 20128766, 20129972, 20133880, 20137842, 20141996, 20151258, 20155561, 20161945, 20164162, 20169453, 20172147, 20176054, 20176105, 20176155, 20176277, 20181251, 20204106, 20221396, 20238991, 20239051, 20239412, 20244047, 20252879, 20252882, 20252885, 20253800, 20254453, 20255603, 20256993, 20257043, 20257143, 20298837, 20298927, 20299256, 20299805, 20302512, 20351734, 20352591, 20352592, 20398634, 20431249, 20447244, 20450830, 20468070, 20472833, 20479370, 20483164, 20483866, 20493555, 20497989, 20504122, 20504833, 20504920, 20506421, 20506433, 20510062, 20519204, 20521341, 20529508, 20533122, 20538449, 20545348, 20545517, 20545527, 20545585, 20560720, 20563329, 20563437, 20563573, 20563622, 20563856, 20566296, 20567144, 20571580, 20571910, 20571914, 20576149, 20576753, 20577945, 20584912, 20592302, 20592843, 20596846, 20621519, 20622378, 20622379, 20622380, 20622381, 20622382, 20622383, 20622411, 20622437, 20622465, 20622525, 20623123, 20623138, 20623196, 20623201, 20623237, 20623262, 20623263, 20623301, 20627026, 20627649, 20634891, 20637820, 20637831, 20637838, 20637844, 20637899, 20637922, 20640335, 20643272, 20643998, 20644258, 20644606, 20644941, 20645091, 20648656, 20648679, 20649299, 20654837, 20658077, 20658421, 20664023, 20664131, 20671919, 20672031, 20672040, 20672042, 20672086, 20672143, 20672162, 20691562, 20703641, 20703660, 20703802, 20703825, 20704077, 20716070, 20718938, 20721557, 20724680, 20743378, 20743398, 20743783, 20749791, 20766777, 20781130, 20784536, 20786534, 20831058, 20870139, 20870448, 20870450, 20874920, 20876294, 20879218, 20879782, 20879783, 20879792, 20879820, 20880345, 20918811, 20923019, 20931970, 20932578, 20932579, 20939176, 20946791, 20952863, 20954390, 20954497, 20957047, 20962267, 20965512, 20973468, 20976603, 20978051, 21099696, 21119694, 21119737, 21119824, 21119825, 21119839, 21182920, 21186925, 21229733, 21252219, 21256164, 21277771, 21302656, 21331501, 21337339, 21363124, 21363142, 21363151, 21363156, 21363193, 21363270, 21363313, 21363453, 21363912, 21366183, 21409822, 21473465, 21475231)

(27099,
 54231,
 55024,
 82774,
 128569,
 171231,
 178521,
 191308,
 315995,
 366834,
 386103,
 393098,
 393359,
 408769,
 507262,
 561750,
 563706,
 563947,
 593765,
 604227,
 604538,
 643900,
 673875,
 676762,
 918955,
 1050248,
 1210319,
 1217047,
 1521733,
 1535264,
 1639773,
 1725054,
 1727004,
 1769935,
 1770239,
 1778694,
 1779108,
 1793105,
 1803680,
 1803854,
 1902578,
 1915192,
 1935178,
 1957873,
 1964962,
 1998982,
 2022154,
 2052228,
 2111087,
 2130214,
 2132030,
 2150621,
 2187425,
 2204318,
 2245758,
 2248202,
 2280594,
 2327399,
 2327410,
 2327498,
 2327521,
 2329822,
 2330248,
 2331874,
 2385561,
 2405982,
 2589926,
 2593307,
 2605569,
 2623654,
 2624064,
 2732604,
 2757453,
 2783807,
 2789434,
 2789436,
 2803704,
 2815311,
 2815313,
 2816546,
 2819243,
 2824952,
 2828866,
 2862016,
 2869420,
 2874300,
 2913188,
 2913194,
 2913504,
 2914467,
 2947555,
 2975705,
 3042083,
 3044490,
 3046993,
 3050032,
 3112087,
 3113029,
 3115457,
 3116033,
 3116122,
 3117261,
 3117282,

In [ ]:
COPY
(
    SELECT *
    FROM read_parquet("/data/processing/partitioned-ohsome-parquet-100997/*/*/*.parquet", hive_partitioning=1)
    WHERE year = 2023 and month=9
)
TO '/data/processing/ohsome_contributions_2023_09.parquet' (FORMAT PARQUET);


COPY
(
    SELECT *
    FROM read_parquet("/data/processing/partitioned-ohsome-parquet-100997/*/*/*.parquet", hive_partitioning=1)
    WHERE year = 2023 and month=10
)
TO '/data/processing/ohsome_contributions_2023_10.parquet' (FORMAT PARQUET);


COPY
(
    SELECT *
    FROM read_parquet("/data/processing/partitioned-ohsome-parquet-100997/*/*/*.parquet", hive_partitioning=1)
    WHERE year = 2023 and month=11
)
TO '/data/processing/ohsome_contributions_2023_11.parquet' (FORMAT PARQUET);
